### Improving Pandas Excel Output

Why - Pandas使輸出DataFrame到Excel非常容易。然而，在自定義輸出和使用Excel的功能使你的輸出盡可能有用方面，選擇有限。

What - 使用優秀的XlsxWriter模塊來定制和增強由熊貓to_excel函數創建的Excel工作簿是很容易的。本文將描述如何使用XlsxWriter和Pandas來製作複雜的、具有視覺吸引力的、有用的Excel工作簿。作為額外的收穫，文章將簡要討論在pandas 0.16.0中引入的新assign函數的使用。

- https://pbpython.com/improve-pandas-excel-output.html

In [1]:
import pandas as pd
import numpy as np
from xlsxwriter.utility import xl_rowcol_to_cell

df = pd.read_excel("data/df-comp-datav2.xlsx")
df.head()

,account,name,street,city,state,post-code,quota,Jan,Feb,Mar
0,211829,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,TX,28752,110000,10000,62000,35000
1,320563,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NC,38365,150000,95000,45000,35000
2,648336,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,IA,76517,300000,91000,120000,35000
3,109996,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,ME,46021,180000,45000,120000,10000
4,121213,Bauch-Goldner,7274 Marissa Common,Shanahanchester,CA,49681,300000,162000,120000,35000


In [2]:
# Add some summary data using the new assign functionality in pandas 0.16
df = df.assign(total=(df['Jan'] + df['Feb'] + df['Mar']))
df = df.assign(quota_pct=(1+(df['total'] - df['quota'])/df['quota']))

新增了 total 和 quota_pct 後你會注意到有幾件事情。

- 列的寬度使它很難看到所有的數據
- 銷售額和百分比沒有以美元或百分比的形式表示。
- 沒有總的信息。
- 總的來說，它是相當無聊的。

In [3]:
writer_orig = pd.ExcelWriter('data/df-comp-datav2-Basic-out.xlsx', engine='xlsxwriter')
df.to_excel(writer_orig, index=False, sheet_name='report')
writer_orig.save()

###  Complex Output 複雜的輸出

只需多寫一點代碼，我們就可以創建一個更複雜的輸出。
我們創建另一個寫作者並使用to_excel來創建我們的工作簿。
關鍵是要獲得對工作表的訪問權，使我們能夠使用該庫中所有的XlsxWriter能力。

In [4]:
writer = pd.ExcelWriter('data/df-comp-datav2-Complex-out.xlsx', engine='xlsxwriter')
df.to_excel(writer, index=False, sheet_name='report')

workbook = writer.book
worksheet = writer.sheets['report']

現在我們有了工作表，我們可以做任何xlsxwriter支持的事情。我鼓勵你看一下XlsxWriter的文檔。它們寫得非常好，向你展示了定制Excel輸出的所有功能。通過訪問如上所示的工作表，你可以輕鬆地將XlsxWriter的所有功能放入你的輸出中。

In [5]:
# 首先，我們通過調整縮放來調整工作表的大小。
worksheet.set_zoom(90)

我們最大的一些改進是通過格式化列來使數據更易讀。 add_format對於改進你的標準輸出非常有用。下面是兩個數字格式化的例子。

In [6]:
# 為有錢的單元格添加一個數字格式。
money_fmt = workbook.add_format({'num_format': '$#,##0', 'bold': True})

# 添加帶有1個小數點的百分比格式
percent_fmt = workbook.add_format({'num_format': '0.0%', 'bold': True})

這個例子顯示瞭如何添加額外的格式，包括底線。

In [7]:
# 總計格式化
total_fmt = workbook.add_format({'align': 'right', 'num_format': '$#,##0',
                                 'bold': True, 'bottom':6})
# 總的百分比格式
total_percent_fmt = workbook.add_format({'align': 'right', 'num_format': '0.0%',
                                         'bold': True, 'bottom':6})

使用set_column改變幾列的大小。這也可以用來對某一列施加格式化。

In [8]:
# 賬戶信息列
worksheet.set_column('B:D', 20)
# 國家列
worksheet.set_column('E:E', 5)
# 郵政代碼
worksheet.set_column('F:F', 10)

0

除了改變G-L的大小之外，我們還可以在整個列上應用貨幣和百分比格式。

In [9]:
# 每月一列
worksheet.set_column('G:K', 12, money_fmt)
# 配額百分比列
worksheet.set_column('L:L', 12, percent_fmt)

0

下一節在我們的數據底部添加了一個總數。在Excel工作中，最大的挑戰是數字索引和單元格標籤之間的轉換。這個循環顯示瞭如何以數字方式循環瀏覽各列，同時使用xl_rowcol_to_cell來獲得單元格的位置。

In [10]:
# 增加總行數
number_rows = len(df.index)

for column in range(6, 11):
    # Determine where we will place the formula
    cell_location = xl_rowcol_to_cell(number_rows+1, column)
    # Get the range to use for the sum formula
    start_range = xl_rowcol_to_cell(1, column)
    end_range = xl_rowcol_to_cell(number_rows, column)
    # Construct and write the formula
    formula = "=SUM({:s}:{:s})".format(start_range, end_range)
    worksheet.write_formula(cell_location, formula, total_fmt)

除了寫總標籤外，我們還想看看我們的配額百分比總和是多少。我們構建一個字符串來計算配額的百分比，並使用write_formula將其寫出來


In [11]:
# Add a total label
worksheet.write_string(number_rows+1, 5, "Total",total_fmt)
percent_formula = "=1+(K{0}-G{0})/G{0}".format(number_rows+2)
worksheet.write_formula(number_rows+1, 11, percent_formula, total_percent_fmt)

0

最後要添加的項目是突出顯示前5個值和後5個值的能力。這為我們提供了一個很好的視覺表現，說明我們可能需要進一步檢查的地方。

首先，我們定義我們想要格式化的範圍。

In [12]:
# Define our range for the color formatting
color_range = "L2:L{}".format(number_rows+1)

In [13]:
# Add a format. Light red fill with dark red text.
format1 = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

# Add a format. Green fill with dark green text.
format2 = workbook.add_format({'bg_color': '#C6EFCE',
                               'font_color': '#006100'})

In [14]:
# Highlight the top 5 values in Green
worksheet.conditional_format(color_range, {'type': 'top',
                                           'value': '5',
                                           'format': format2})

0

In [15]:
# Highlight the bottom 5 values in Red
worksheet.conditional_format(color_range, {'type': 'bottom',
                                           'value': '5',
                                           'format': format1})
writer.save()